# `Enum` y `dataclass`es 

Vamos a ver ahora dos tipos de datos que pueden ser útiles más allá de los objetos que uno pueda definir en Python mediante clases. Ambos tipos de datos se relacionan con la _inmutabilidad_, propiedad que tiene muchos casos de uso relevantes y es de mucha ayuda para crear código robusto.

## `Enum`s

Los `enum`s (enumeraciones) son una forma de asociar simbólicamente un conjunto de etiquetas a un conjunto de valores constantes, y se introducen en Python con la versión 3.4. Los `enum` modelan un conjunto _limitado_ de valores que una variable puede tomar, y donde cada valor tiene un nombre descriptivo.

Para definir un `enum`, es necesario importar la clase `Enum` del módulo correspondiente

In [1]:
from enum import Enum 

In [2]:
class ColorCMYK(Enum):
    YELLOW = 1
    CYAN = 2
    MAGENTA = 3
    BLACK = 4

En este caso hemos definido un `enum` con tres elementos correspondientes a cuatro colores.

In [3]:
def print_color(color: ColorCMYK) -> None:        

    print(f"Color  : {color}")    
    print(f"Nombre : {color.name}" )
    print(f"Valor  : {color.value}" )


In [4]:
print_color(ColorCMYK.YELLOW)

Color  : ColorCMYK.YELLOW
Nombre : YELLOW
Valor  : 1


> Atención: Por **convención** se usan MAYÚSCULAS para las opciones que puede tener un Enum, al igual que en otros lenguajes de programación donde también se estila usarlas para las constantes.

Efectivamente los valores del Enum son constantes y no es posible reasignarlos:

In [5]:
ColorCMYK.YELLOW = 42

AttributeError: cannot reassign member 'YELLOW'

In [6]:
ColorCMYK.YELLOW.value = 4

AttributeError: <enum 'Enum'> cannot set attribute 'value'

In [7]:
class ColorRGB(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3
    
    def __repr__(self):        
        return f"Color  : {self}\nNombre : {self.name}\nValor  : {self.value}\n"


In [8]:
ColorRGB.RED

Color  : ColorRGB.RED
Nombre : RED
Valor  : 1

Se pueden comparar distintos enums:

In [9]:
ColorRGB.RED == ColorCMYK.YELLOW

False

In [10]:
ColorRGB.RED.value == ColorCMYK.YELLOW.value

True

In [11]:
print(ColorRGB.RED == ColorCMYK.YELLOW)
print(ColorRGB.RED is ColorCMYK.YELLOW)

False
False


### Enums y `match` 

Una estructura de control introducida en Python 3.10 es `match-case`, y puede ser interesante de usar junto con Enums. El `match-case` fue un pedido recurrente de la comunidad para poseer una estructura de control de flujo múltiple más clara que el `if-elif-else`. Se comporta en forma similar a los `switch` que usan otros lenguajes de programación. La estructura que tiene es la siguiente:

```python
match variable:
    case patrón1:
        # Código para patrón1
    case patrón2:
        # Código para patrón2
    ...        
    case _:
        # Código para el caso por defecto
```

Por ejemplo:

In [12]:
def describe_color(color):
    match color:
        case ColorCMYK.YELLOW:
            return "Amarillo"
        case ColorCMYK.CYAN:
            return "Cian"
        case ColorCMYK.MAGENTA:
            return "Magenta"
        case ColorRGB.RED:
            return "Rojo"
        case ColorRGB.GREEN:
            return "Verde"
        case ColorRGB.BLUE:
            return "Azul"
        case _:
            return "Color no reconocido"

print(describe_color(ColorCMYK.YELLOW))
print(describe_color(ColorRGB.RED))
print(describe_color(ColorRGB.GREEN))
print(describe_color("Negro"))

Amarillo
Rojo
Verde
Color no reconocido


La estructura `match-case` acepta patrones avanzados, comparando estructuras más complejas:

In [13]:
def detecta_coordenadas(coord):
    match coord:
        case (0, 0):
            return "Origen"
        case (x, 0):
            return f"En el eje X, en {x}"
        case (0, y):
            return f"En el eje Y, en {y}"
        case (x, y):
            return f"En el plano: ({x}, {y})"
        case _:
            return "Coordenada no válida"
        
print(detecta_coordenadas((0, 5)))  # "En el eje Y, en 5"
print(detecta_coordenadas("cero, cero"))

En el eje Y, en 5
Coordenada no válida


In [14]:
def clasifica_lista(lista):
    match lista:
        case []: 
            print("Lista vacía")
            return None 
        case [x]:  # Coincide con una lista de un solo elemento
            print (f"Lista con un solo elemento: {x}")
            return x
        case [x, y]:  # Coincide con una lista de dos elementos
            print (f"Lista con dos elementos: {x} y {y}")
            return (x,y)
        case [x, y, *resto]:  # Coincide con una lista de tres o más elementos
            print (f"Lista con tres o más elementos: {x}, {y}, y otros {len(resto)} elementos")
            return resto
        case _:  # Coincide con cualquier otro caso
            print ("Lista vacía o no reconocida")
            return

# Probar con diferentes listas
clasifica_lista([10])         
clasifica_lista([10, 20])     
clasifica_lista([10, 20, 30]) 
clasifica_lista([10, 20, 30, 40]) 
clasifica_lista([])      
clasifica_lista("Hola")     


Lista con un solo elemento: 10
Lista con dos elementos: 10 y 20
Lista con tres o más elementos: 10, 20, y otros 1 elementos
Lista con tres o más elementos: 10, 20, y otros 2 elementos
Lista vacía
Lista vacía o no reconocida


In [15]:
v = clasifica_lista([10, 20, 30, 40])
print(v,type(v))

v = clasifica_lista([ColorCMYK.BLACK, ColorRGB.RED])
print(v,type(v))

Lista con tres o más elementos: 10, 20, y otros 2 elementos
[30, 40] <class 'list'>
Lista con dos elementos: ColorCMYK.BLACK y ColorRGB.RED
(<ColorCMYK.BLACK: 4>, Color  : ColorRGB.RED
Nombre : RED
Valor  : 1
) <class 'tuple'>


Los distintos casos posibles aceptan el operador `|` que se usa para agruparlos:

In [16]:
from enum import Enum

# Definimos un Enum para los días de la semana
class Dia(Enum):
    LUNES = 1
    MARTES = 2
    MIERCOLES = 3
    JUEVES = 4
    VIERNES = 5
    SABADO = 6
    DOMINGO = 7

    # Función para determinar si es día laboral o fin de semana
    def es_dia_laboral(self):
        match self:
            case Dia.LUNES | Dia.MIERCOLES:
                return "Tengo clases de Python 🥳"
            case  Dia.MARTES | Dia.JUEVES | Dia.VIERNES :
                return "Es un día laboral 🧐"
            case Dia.SABADO | Dia.DOMINGO:
                return "Es fin de semana 😆"
            case _:
                return "Día no válido"

# Probar con diferentes días
print(Dia.LUNES.es_dia_laboral())   
print(Dia.SABADO.es_dia_laboral())  
print(Dia.MIERCOLES.es_dia_laboral())


Tengo clases de Python 🥳
Es fin de semana 😆
Tengo clases de Python 🥳


O comparar tipos de datos

In [17]:
def printer_color(color: ColorRGB | ColorCMYK) -> None:
    match color:
        case ColorRGB():
            print(f"Usando RGB: {color}\n")
        case ColorCMYK():
            print(f"Usando YMgCy: {color}\n")
        case _:
            print("Color no reconocido\n")

printer_color(ColorRGB.RED)
printer_color(ColorCMYK.YELLOW)
printer_color("Negro")

Usando RGB: ColorRGB.RED

Usando YMgCy: ColorCMYK.YELLOW

Color no reconocido



Es posible hacer comparaciones más complejas todavía, por ejemplo, usando clases:

In [18]:
class Persona:
    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad

def saluda_a(persona):
    match persona:
        case Persona() if persona.edad >= 18:
            return f"Hola {persona.nombre}, eres mayor de edad."
        case Persona() if 16 <= persona.edad < 18:
            return f"Hola {persona.nombre}, podés manejar pero no comprar alcohol"
        case Persona():
            return f"Hola {persona.nombre}, eres menor de edad."
        case _:
            return "Eres un alien"
        
print(saluda_a(Persona("Juan",12))) 
print(saluda_a(Persona("Ana",19)))  
print(saluda_a(Persona("Mabel",17)))
print(saluda_a("Chewbacca"))

Hola Juan, eres menor de edad.
Hola Ana, eres mayor de edad.
Hola Mabel, podés manejar pero no comprar alcohol
Eres un alien


Otra forma de hacerlo es a través del denominado match _posicional_. Para ello se agrega el atributo  __match_args__ a la clase, que contiene una tupla que representa los argumentos de creación de la clase tal como figuran en el `__init__`. 

> Atención: consultar [la ayuda](https://peps.python.org/pep-0636/) para comprender en profundidad cómo funciona el `match-case` cuando se comparan estructuras de datos complejas como las clases.

In [19]:
class Persona:
    __match_args__ = ("nombre","edad")

    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad

def saluda_a(persona):
    match persona:
        case Persona(nombre, edad) if edad >= 18:
            return f"Hola {nombre}, eres mayor de edad."
        case Persona(nombre, edad) if 16 <= edad < 18:
            return f"Hola {nombre}, podés manejar pero no comprar alcohol"
        case Persona(nombre, edad):
            return f"Hola {nombre}, eres menor de edad."
        case _:
            return "Eres un alien"
        
print(saluda_a(Persona("Juan",12))) 
print(saluda_a(Persona("Ana",19)))  
print(saluda_a(Persona("Mabel",17)))
print(saluda_a("Chewbacca"))

Hola Juan, eres menor de edad.
Hola Ana, eres mayor de edad.
Hola Mabel, podés manejar pero no comprar alcohol
Eres un alien


## Dataclasses

En muchísimas situaciones uno necesita utilizar una clase con ciertos métodos habituales, como un constructor default. Para ello Python provee un módulo que define un decorador `@dataclass` que los genera. 

In [20]:
from dataclasses import dataclass

@dataclass
class Atomo:
    nombre: str    
    simbolo: str
    N: int # número atómico
    A: int # número de masa


In [21]:
hidrogeno = Atomo("Hidrógeno", "H", 1, 1)
helio = Atomo("Helio", "He", 2, 4)

print(hidrogeno)
hidrogeno

Atomo(nombre='Hidrógeno', simbolo='H', N=1, A=1)


Atomo(nombre='Hidrógeno', simbolo='H', N=1, A=1)

Entre los métodos que el decorador genera automáticamente están el constructor `__init__`, los métodos `__repr__` y `__str__` y el método `__eq__` que provee igualdad estructural:

In [22]:
h = Atomo("Hidrógeno", "H", 1, 1)
print(h==hidrogeno)
print(h is hidrogeno)

True
False


Además de la sintaxis sencilla, se pueden crear dataclasses con argumentos default:

In [23]:
class StockStatus(Enum):
    DISPONIBLE = "En stock"
    AGOTADO = "Sin stock"
    QUEDAN_POCOS = "Stock bajo!" 


@dataclass
class Producto:
    nombre: str
    precio: float
    stock: StockStatus = StockStatus.AGOTADO 

p = Producto("Laptop", 1000.0)
print(p)    

b = Producto("Cerveza",2.5, StockStatus.DISPONIBLE)
print(b)

Producto(nombre='Laptop', precio=1000.0, stock=<StockStatus.AGOTADO: 'Sin stock'>)
Producto(nombre='Cerveza', precio=2.5, stock=<StockStatus.DISPONIBLE: 'En stock'>)


Para finalizar, es posible poblar una `dataclass` a partir de un diccionario en forma sencilla, siempre y cuando las claves del diccionario se correspondan unívocamente con los campos de la estructura de la `dataclass`

In [24]:
cerveza = { "nombre": "Cerveza", "precio": 2.5, "stock": StockStatus.DISPONIBLE } 

b = Producto(**cerveza)
print(b)

Producto(nombre='Cerveza', precio=2.5, stock=<StockStatus.DISPONIBLE: 'En stock'>)


In [25]:
b.nombre = "Cerveza artesanal"
print(b)

Producto(nombre='Cerveza artesanal', precio=2.5, stock=<StockStatus.DISPONIBLE: 'En stock'>)


In [26]:
b.nombre = 4

Otra propiedad interesante que poseen las `dataclass`es consiste en utilizar el argumento `frozen` para evitar que los objetos sean modificados una vez creados. Si intentamos modificar un atributo de un objeto `frozen`, se lanzará una excepción `FrozenInstanceError`.

In [27]:
@dataclass(frozen=True)
class Atomo():
    nombre: str    
    simbolo: str
    N: int # número atómico
    A: int # número de masa


In [28]:
Ca = Atomo("Calcio", "Ca", 20, 40)
print(Ca)
Ca.A = 14

Atomo(nombre='Calcio', simbolo='Ca', N=20, A=40)


FrozenInstanceError: cannot assign to field 'A'

-----

## Ejercicios 08 (b)

2. El archivo `atomos_t.json` contiene datos atómicos y físicos de los primeros átomos de la tabla periódica. Se puede usar el módulo `json` para leer este archivo de la siguiente manera
   ```python
   import json
   with open('atomos_t.json', 'r') as file:  # Verifique que el path al archivo sea el correcto en su caso
	   atomos = json.load(file)
   ```

   De esta manera se crea un diccionario `atomos` con la información del archivo.

   - Cree una `dataclass` para manejar los datos atómicos, que incluya el nombre del elemento, el símbolo, el número atómico y la masa atómica.
   - Extienda la clase anterior para poder manejar el estado del material a temperatura ambiente ('State at Room Temp'). Para ello cree un `enum` adecuado para representarlo y construya una nueva `dataclass` adecuada.
   - Modifique `__repr__` y `__str__` para que se imprima la información de cada átomo en forma clara y bella.
   - ¿Qué estrategia/s usaría para incorporar las temperaturas de fusión ('Melting Point') y de ebullición ('Boiling Point') de los átomos de la lista?


-----
